In [1]:
# Dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as sts
import os
import json
import requests 

This section will make API call and create dataframe with needed data from the Urban Institute of Education Portal accessing the Directory of Common Core of Data from the US Department of Education which is the primary database on public elementary and secondary education. Specifically pulling 2014 directory data for school districts in the state of Florida which based on the api documentation the state code is found under the fips search key at https://educationdata.urban.org/documentation/school-districts.html#overview

In [2]:
# Build query URL
url = "https://educationdata.urban.org/api/v1/school-districts/ccd/directory/"
year = "2014/"
state_code = "12"

query_url = f"{url}{year}?fips={state_code}"

# Request data in json and store variable
fl_ccd_directory_response = requests.get(query_url)
fl_ccd_directory_data = fl_ccd_directory_response.json()

# Establish the results from the request to loop through below
results_directory = fl_ccd_directory_data['results']

# Create Lists to store values for the columns
district_name = []
dir_nces_id = []
state_id = []
latitude = []
longitude = []
urban_locale = []
agency_type = []
total_schools = []
total_students = []
spec_ed_stud = []
eng_lang_stud = []
teachers_fte = []
instru_aids_fte = []
guidance_cons_fte = []
school_staff_fte = []
district_staff_fte = []
staff_total_fte = []

# Loop through results and store values in appropriate lists
for result in results_directory:
    district_name.append(result['lea_name'])
    dir_nces_id.append(result['leaid'])
    state_id.append(result['state_leaid'])
    latitude.append(result['latitude'])
    longitude.append(result['longitude'])
    urban_locale.append(result['urban_centric_locale'])
    agency_type.append(result['agency_type'])
    total_schools.append(result['number_of_schools'])
    total_students.append(result['enrollment'])
    spec_ed_stud.append(result['spec_ed_students'])
    eng_lang_stud.append(result['english_language_learners'])
    teachers_fte.append(result['teachers_total_fte'])
    instru_aids_fte.append(result['instructional_aides_fte'])
    guidance_cons_fte.append(result['guidance_counselors_total_fte'])
    school_staff_fte.append(result['school_staff_total_fte'])
    district_staff_fte.append(result['lea_staff_total_fte'])
    staff_total_fte.append(result['staff_total_fte'])
    
# Create dataframe from results
fl_ccd_directory = pd.DataFrame({"District Name":district_name, "NCES ID":dir_nces_id, "State ID":state_id,                          
                                 "Degree of urbanization":urban_locale, "Agency Type":agency_type,
                                "Total Schools":total_schools, "Total Students Enrolled":total_students,
                                "Special Ed Students":spec_ed_stud, "English Language Students":eng_lang_stud,
                                "Total FTE Teachers":teachers_fte, "Total FTE Instructional Aids": instru_aids_fte,
                                "Total FTE Guidance Counselors": guidance_cons_fte, "Total FTE School Staff":school_staff_fte,
                                "Total FTE District Staff": district_staff_fte, "Total FTE Staff": staff_total_fte,
                                 "Latitude":latitude, "Longitude":longitude})

# Remove all districts that are not regular local school district
clean_fl_ccd_directory = fl_ccd_directory.loc[fl_ccd_directory['Agency Type'] == 1]


clean_fl_ccd_directory

,District Name,NCES ID,State ID,Degree of urbanization,Agency Type,Total Schools,Total Students Enrolled,Special Ed Students,English Language Students,Total FTE Teachers,Total FTE Instructional Aids,Total FTE Guidance Counselors,Total FTE School Staff,Total FTE District Staff,Total FTE Staff,Latitude,Longitude
2,ALACHUA,1200030,01,12,1,73,28689,3788,584,1833,535,61,None,None,4249,29.652601,-82.318977
3,BAKER,1200060,02,32,1,9,4936,538,11,301,65,10,None,None,604,30.277151,-82.118149
5,BAY,1200090,03,22,1,51,27641,4575,567,1844,732,66,None,None,3735,30.172516,-85.675934
6,BRADFORD,1200120,04,32,1,12,3199,645,6,242,56,8,None,None,507,29.951584,-82.112663
7,BREVARD,1200150,05,21,1,129,72285,11931,2344,4776,791,183,None,None,9109,28.248600,-80.736115
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,UNION,1201890,63,32,1,9,2384,387,0,175,32,6,None,None,343,30.017477,-82.338341
66,VOLUSIA,1201920,64,21,1,92,61777,9879,3479,4153,593,142,None,None,7586,29.030180,-81.308472
67,WAKULLA,1201950,65,42,1,15,5141,976,8,327,84,9,None,None,708,30.176279,-84.379211
68,WALTON,1201980,66,41,1,23,8435,799,300,569,99,18,None,None,1164,30.720671,-86.098549


In [3]:
# Create a list of district names that were not regular local schools to drop from other tables
drop_agencies = fl_ccd_directory[fl_ccd_directory['Agency Type'] != 1]
drop_ids = drop_agencies['District Name'].tolist()

drop_ids

['FL VIRTUAL',
 'CESA',
 'LAKE WALES CHARTER SCHOOLS',
 'WASH SPECIAL',
 'FAU LAB SCH',
 'FSU LAB SCH',
 'FAMU LAB SCH',
 'UF LAB SCH',
 'DEAF/BLIND']

This section will make API call and create dataframe with needed data from the Urban Institute of Education Portal accessing the Directory of Common Core of Data from the US Department of Education which is the primary database on public elementary and secondary education. Specifically pulling 2014 financal data for school districts in the state of Florida.  Use the same fips code for state above.

In [4]:
# Build query URL
url = "https://educationdata.urban.org/api/v1/school-districts/ccd/finance/"
year = "2014/"
state_code = "12"

query_url = f"{url}{year}?fips={state_code}"

# Request data in json and store variable
fl_ccd_finance_response = requests.get(query_url)
fl_ccd_finance_data = fl_ccd_finance_response.json()

# Establish the results from the request to loop through below
results_financial = fl_ccd_finance_data['results']

# Create Lists to store values for the columns
fin_nces_id = []
total_rev = []
fed_total_rev = []
st_total_rev = []
loc_total_rev = []
loc_prop_tax_rev = []
total_exp = []
intruct_staff_exp = []
general_admin_exp = []
school_admin_exp = []
operation_plant_exp = []
student_transport_exp = []

# Loop through results and store values in appropriate lists
for result in results_financial:
    fin_nces_id.append(result['leaid'])
    total_rev.append(result['rev_total'])
    fed_total_rev.append(result['rev_fed_total'])
    st_total_rev.append(result['rev_state_total'])
    loc_total_rev.append(result['rev_local_total'])
    loc_prop_tax_rev.append(result['rev_local_prop_tax'])
    total_exp.append(result['exp_total'])
    intruct_staff_exp.append(result['exp_current_instruc_staff'])
    general_admin_exp.append(result['exp_current_general_admin'])
    school_admin_exp.append(result['exp_current_sch_admin'])
    operation_plant_exp.append(result['exp_current_operation_plant'])
    student_transport_exp.append(result['exp_current_student_transport'])
    
# Create dataframe from results
fl_ccd_financial = pd.DataFrame({"NCES ID":fin_nces_id, "Total Revenue":total_rev, "Total Federal Revenue":fed_total_rev,                        
                                 "Total State Revenue":st_total_rev, "Total Local Revenue":loc_total_rev,
                                "Local Revenue (Property Tax)":loc_prop_tax_rev, "Total Expense":total_exp,
                                 "Total Expense for Instruction Staff":intruct_staff_exp,
                                "Total Expense for General Admin":general_admin_exp, "Total Expense for School Admin":school_admin_exp,
                                "Total Expense for Operations":operation_plant_exp, "Total Expense for Student Transportation":student_transport_exp})

# Remove all districts without financial data
clean_fl_ccd_financial = fl_ccd_financial.loc[fl_ccd_financial['Total Revenue'] > 0]

clean_fl_ccd_financial.head()

,NCES ID,Total Revenue,Total Federal Revenue,Total State Revenue,Total Local Revenue,Local Revenue (Property Tax),Total Expense,Total Expense for Instruction Staff,Total Expense for General Admin,Total Expense for School Admin,Total Expense for Operations,Total Expense for Student Transportation
2,1200030,272451000.0,34881000.0,119810000.0,117760000,103083000,269955000,21763000,3041000,13128000,29535000,11174000
3,1200060,42674000.0,4704000.0,29197000.0,8773000,6266000,44360000,2595000,732000,2007000,4385000,2613000
5,1200090,268131000.0,26772000.0,103527000.0,137832000,101316000,278559000,11183000,2045000,13550000,20567000,8132000
6,1200120,31053000.0,4941000.0,17021000.0,9091000,6476000,31823000,1490000,734000,1792000,3197000,1629000
7,1200150,673236000.0,76493000.0,320008000.0,276735000,221801000,652911000,49626000,5282000,36220000,55676000,17750000


This section will make API call and create dataframe from the Urban Institute of Education Portal accessing The US Census Bureau's Small Area Income and Poverty Estimates.  Specifically pulling 2014 estimated poverity data for school aged children fo school districts in the state of Florida.  Use the same fips paramaters as above.

In [5]:
# Build query URL
url = "https://educationdata.urban.org/api/v1/school-districts/saipe/"
year = "2014/"
state_code = "12"

query_url = f"{url}{year}?fips={state_code}"

# Request data in json and store variable
fl_saipe_poverty_response = requests.get(query_url)
fl_saipe_poverty_data = fl_saipe_poverty_response.json()

# Establish the results from the request to loop through below
results_poverity = fl_saipe_poverty_data['results']

# Create Lists to store values for the columns
pov_nces_id = []
est_pop_total = []
est_pop_5to17 = []
est_pop_5to17_perc = []
est_pop_5to17_poverity = []
est_pop_5to17_poverity_perc = []


# Loop through results and store values in appropriate lists
for result in results_poverity:
    pov_nces_id.append(result['leaid'])
    est_pop_total.append(result['est_population_total'])
    est_pop_5to17.append(result['est_population_5_17'])
    est_pop_5to17_perc.append(result['est_population_5_17_pct'])
    est_pop_5to17_poverity.append(result['est_population_5_17_poverty'])
    est_pop_5to17_poverity_perc.append(result['est_population_5_17_poverty_pct'])

    
# Create dataframe from results
fl_saipe_poverity = pd.DataFrame({"NCES ID":pov_nces_id, "Estimated Total Population":est_pop_total, 
                                 "Estimated Total Population Ages 5–17":est_pop_5to17,                        
                                 "Share of Population that are school ages 5–17":est_pop_5to17_perc, 
                                 "Estimated Population Ages 5–17 in poverity":est_pop_5to17_poverity,
                                "Share of School-Age Population (5–17) in poverity":est_pop_5to17_poverity_perc})

fl_saipe_poverity.head()

,NCES ID,Estimated Total Population,Estimated Total Population Ages 5–17,Share of Population that are school ages 5–17,Estimated Population Ages 5–17 in poverity,Share of School-Age Population (5–17) in poverity
0,1200030,256380.0,32012.0,0.124862,6438.0,0.201112
1,1200060,27093.0,5004.0,0.184697,1146.0,0.229017
2,1200090,178985.0,27626.0,0.154348,5939.0,0.214979
3,1200120,26702.0,3847.0,0.144072,1047.0,0.272160
4,1200150,556885.0,78183.0,0.140393,15431.0,0.197370


In [6]:
# Combined all above API calls in to one dataframe to match with District Name

dem_pov_nces_combined = pd.merge(clean_fl_ccd_directory,fl_saipe_poverity, on="NCES ID")
fl_nces_combined = pd.merge(dem_pov_nces_combined,clean_fl_ccd_financial, on="NCES ID")
fl_nces_combined.head()

,District Name,NCES ID,State ID,Degree of urbanization,Agency Type,Total Schools,Total Students Enrolled,Special Ed Students,English Language Students,Total FTE Teachers,...,Total Federal Revenue,Total State Revenue,Total Local Revenue,Local Revenue (Property Tax),Total Expense,Total Expense for Instruction Staff,Total Expense for General Admin,Total Expense for School Admin,Total Expense for Operations,Total Expense for Student Transportation
0,ALACHUA,1200030,01,12,1,73,28689,3788,584,1833,...,34881000.0,119810000.0,117760000,103083000,269955000,21763000,3041000,13128000,29535000,11174000
1,BAKER,1200060,02,32,1,9,4936,538,11,301,...,4704000.0,29197000.0,8773000,6266000,44360000,2595000,732000,2007000,4385000,2613000
2,BAY,1200090,03,22,1,51,27641,4575,567,1844,...,26772000.0,103527000.0,137832000,101316000,278559000,11183000,2045000,13550000,20567000,8132000
3,BRADFORD,1200120,04,32,1,12,3199,645,6,242,...,4941000.0,17021000.0,9091000,6476000,31823000,1490000,734000,1792000,3197000,1629000
4,BREVARD,1200150,05,21,1,129,72285,11931,2344,4776,...,76493000.0,320008000.0,276735000,221801000,652911000,49626000,5282000,36220000,55676000,17750000


This section will read in the excel files from the Florida Department of Education for 2014 and clean the dataframes with only the needed information.

In [7]:
# Read in Florida District 21+ Absents Data
fl_2014_21absents_path = "fldoe_data/2014_FL_21absents.xlsx"
fl_2014_21absents_data = pd.read_excel(os.path.join(fl_2014_21absents_path), header=2)

# Remove all district names that were dropped due to not local school district
clean_fl_2014_21absents=fl_2014_21absents_data[~fl_2014_21absents_data["District Name"].isin(drop_ids)]

clean_fl_2014_21absents

,District #,District Name,Enrollments,Absent 21 Days or Over,% Absent 21 or More Days
0,0,FLORIDA,3111840,303913,0.097663
1,1,ALACHUA,32688,3101,0.094867
2,2,BAKER,5377,1005,0.186907
3,3,BAY,32074,4095,0.127674
4,4,BRADFORD,3820,703,0.184031
...,...,...,...,...,...
62,63,UNION,2644,465,0.175870
63,64,VOLUSIA,70784,7841,0.110774
64,65,WAKULLA,5615,989,0.176135
65,66,WALTON,9621,723,0.075148


In [8]:
# Read in Florida District Dropout Data
fl_2014_dropout_path = "fldoe_data/2014_FL_dropout_rates.xls"
fl_2014_dropout_data = pd.read_excel(os.path.join(fl_2014_dropout_path),header=4)

# Remove all district names that were dropped due to not local school district and state total
clean_fl_2014_dropout=fl_2014_dropout_data[~fl_2014_dropout_data["District Name"].isin(drop_ids)]
clean_fl_2014_dropout = clean_fl_2014_dropout.loc[clean_fl_2014_dropout['District Name'] != "FLORIDA"]

clean_fl_2014_dropout

,Dist #,District Name,White Dropouts (numerator),White Enrollment (denominator),White Dropout Rate,Black Dropouts (numerator),Black Enrollment (denominator),Black Dropout Rate,Hispanic Dropouts (numerator),Hispanic Enrollment (denominator),...,Pacific Islander Dropout Rate,American Indian Dropouts (numerator),American Indian Enrollment (denominator),American Indian Dropout Rate,Two or More Races Dropouts (numerator),Two or More Races Enrollment (denominator),Two or More Races Dropout Rate,Total Dropouts (numerator),Total Enrollment (denominator),Total Dropout Rate
1,1,ALACHUA,58,4418,0.0131281,174,3115,0.055859,17,803,...,*,1,20,0.05,14,419,0.0334129,265,9182,0.028861
2,2,BAKER,13,1131,0.0114943,0,151,0.000000,0,24,...,*,*,*,*,0,26,0,13,1344,0.009673
3,3,BAY,65,5713,0.0113776,16,1255,0.012749,8,461,...,*,0,34,0,4,359,0.0111421,95,8033,0.011826
4,4,BRADFORD,2,686,0.00291545,4,210,0.019048,0,24,...,*,*,*,*,0,24,0,6,955,0.006283
5,5,BREVARD,101,14766,0.00684004,48,3130,0.015335,26,3138,...,0,0,55,0,15,1441,0.0104094,191,23071,0.008279
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,63,UNION,3,554,0.00541516,3,115,0.026087,0,36,...,*,*,*,*,0,28,0,6,741,0.008097
64,64,VOLUSIA,109,12604,0.00864805,58,3393,0.017094,32,3757,...,0,4,78,0.0512821,5,734,0.00681199,208,20976,0.009916
65,65,WAKULLA,16,1194,0.0134003,2,157,0.012739,0,37,...,*,*,*,*,0,61,0,18,1468,0.012262
66,66,WALTON,39,1870,0.0208556,3,208,0.014423,12,193,...,*,0,11,0,0,103,0,54,2408,0.022425


In [9]:
# Read in Florida District Demographic Data
fl_2014_demographics_path = "fldoe_data/2014_FL_membership_demographics.xls"
fl_2014_demographics_data = pd.read_excel(os.path.join(fl_2014_demographics_path), sheet_name = "DISTRICT", header=4)

# Remove all district names that were dropped due to not local school district and only selct rows for district totals
clean_fl_2014_demographics=fl_2014_demographics_data[~fl_2014_demographics_data["District"].isin(drop_ids)]
clean_fl_2014_demographics = clean_fl_2014_demographics.loc[clean_fl_2014_demographics['Grade'] == "TOTAL"]

clean_fl_2014_demographics

,District #,District,Grade,White,Black or African American,Hispanic/Latino,Asian,Native Hawaiian or Other Pacific Islander,American Indian or Alaska Native,Two or More Races,Female Total,Male Total,Total
14,1,ALACHUA,TOTAL,12837,10177,2483,1370,28,55,1726,14040,14636,28676
29,2,BAKER,TOTAL,4096,572,92,20,2,14,141,2383,2554,4937
44,3,BAY,TOTAL,19231,4229,1818,580,37,118,1548,13435,14126,27561
59,4,BRADFORD,TOTAL,2269,727,67,25,12,6,80,1533,1653,3186
74,5,BREVARD,TOTAL,45137,10607,9727,1508,92,166,5042,35062,37217,72279
...,...,...,...,...,...,...,...,...,...,...,...,...,...
944,63,UNION,TOTAL,1852,332,87,5,3,6,99,1170,1214,2384
959,64,VOLUSIA,TOTAL,36880,9627,11405,1160,72,175,2457,29931,31845,61776
974,65,WAKULLA,TOTAL,4168,542,150,28,4,14,235,2431,2710,5141
989,66,WALTON,TOTAL,6525,532,856,73,7,27,417,4028,4409,8437


In [10]:
# Read in Florida District Teacher Out of Field Data
fl_2014_teacher_outfield_path = "fldoe_data/2014_FL_teacher_outoffield.xls"
fl_2014_teacher_outfield_data = pd.read_excel(os.path.join(fl_2014_teacher_outfield_path), header=3)

# Remove all district names that were dropped due to not local school district
clean_fl_2014_teacher_outfield = fl_2014_teacher_outfield_data[~fl_2014_teacher_outfield_data["District Name"].isin(drop_ids)]
clean_fl_2014_teacher_outfield = clean_fl_2014_teacher_outfield.loc[clean_fl_2014_teacher_outfield['School Name'] == "DISTRICT TOTAL"]

clean_fl_2014_teacher_outfield

,Dist #,District Name,Sch #,School Name,Total\nCourses,Total\nIn Field,Total\nOut Field,% In Field,% Out Field
1,1,ALACHUA,,DISTRICT TOTAL,34441,34302,139,0.99596,0.00404
66,2,BAKER,,DISTRICT TOTAL,4459,3987,472,0.89415,0.10585
75,3,BAY,,DISTRICT TOTAL,26142,25531,611,0.97663,0.02337
127,4,BRADFORD,,DISTRICT TOTAL,3697,3352,345,0.90668,0.09332
139,5,BREVARD,,DISTRICT TOTAL,92484,90968,1516,0.98361,0.01639
...,...,...,...,...,...,...,...,...,...
4019,63,UNION,,DISTRICT TOTAL,2172,2024,148,0.93186,0.06814
4026,64,VOLUSIA,,DISTRICT TOTAL,79789,76261,3528,0.95578,0.04422
4117,65,WAKULLA,,DISTRICT TOTAL,4417,4093,324,0.92665,0.07335
4131,66,WALTON,,DISTRICT TOTAL,8495,8311,184,0.97834,0.02166


In [11]:
# Read in Florida District Free and Reduced Lunch Data - specific sheet read is the latest survey for the year
#2014-15 SURVEY 3 PRELIMINARY DATA (as of 5/5/2015)
fl_2014_lunch_path = "fldoe_data/2014_FL_lunch_status.xlsx"
fl_2014_lunch_data = pd.read_excel(os.path.join(fl_2014_lunch_path), sheet_name = "Demo Preliminary Survey 3-Dist",
                                  header=4)

# Remove all district names that were dropped due to not local school district
clean_fl_2014_lunch = fl_2014_lunch_data[~fl_2014_lunch_data["District Name"].isin(drop_ids)]

# Groupby District Number
grouped_fl_2014_lunch = clean_fl_2014_lunch.groupby(["District Number","District Name"])

grouped_fl_2014_lunch.sum()

,,White,Unnamed: 4,Unnamed: 5,Black or African American,Unnamed: 7,Unnamed: 8,Hispanic or Latino,Unnamed: 10,Unnamed: 11,Asian,...,Unnamed: 17,American Indian or Alaska Native,Unnamed: 19,Unnamed: 20,Two or More Races,Unnamed: 22,Unnamed: 23,Total,Unnamed: 25,Unnamed: 26
District Number,District Name,,,,,,,,,,,,,,,,,,,,,
1.0,ALACHUA,1697,1928,3625,3945,4080,8025,588,705,1293,126,...,12,14,13,27,481,506,987,6858,7393,14251
2.0,BAKER,1097,1159,2256,237,286,523,27,36,63,6,...,1,5,6,11,60,54,114,1432,1546,2978
3.0,BAY,5024,5449,10473,1807,1774,3581,644,672,1316,148,...,27,41,41,82,542,550,1092,8214,8673,16887
4.0,BRADFORD,717,809,1526,337,312,649,28,29,57,4,...,10,3,3,6,38,31,69,1131,1193,2324
5.0,BREVARD,8846,9554,18400,4152,4216,8368,2940,3104,6044,231,...,54,44,48,92,1401,1512,2913,17632,18701,36333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63.0,UNION,533,559,1092,137,142,279,35,26,61,0,...,3,3,3,6,37,45,82,745,779,1524
64.0,VOLUSIA,9250,10061,19311,4079,4203,8282,4461,4916,9377,261,...,49,66,54,120,916,878,1794,19060,20424,39484
65.0,WAKULLA,984,1090,2074,188,235,423,55,51,106,5,...,0,3,3,6,98,84,182,1333,1467,2800


# Finding the Mean and Median for Dropout Rate

In [79]:
florida_dropout_rate = clean_fl_2014_dropout[["District Name", "Total Dropout Rate"]]
florida_dropout_rate.head()

,District Name,Total Dropout Rate
1,ALACHUA,0.028861
2,BAKER,0.009673
3,BAY,0.011826
4,BRADFORD,0.006283
5,BREVARD,0.008279


In [13]:
florida_dropout_rate.describe()

,Total Dropout Rate
count,67.000000
mean,0.016037
std,0.011796
min,0.000000
25%,0.007894
50%,0.011628
75%,0.022702
max,0.057082


In [14]:
mean = florida_dropout_rate["Total Dropout Rate"].mean()
mean

0.016036537313432835

In [15]:
florida_dropout_rate_mean = florida_dropout_rate.loc[(florida_dropout_rate["Total Dropout Rate"] > mean - .004) & (florida_dropout_rate["Total Dropout Rate"] < mean + .004)]
florida_dropout_rate_mean

,District Name,Total Dropout Rate
6,BROWARD,0.014282
9,CITRUS,0.014999
19,FRANKLIN,0.014837
29,HILLSBOROUGH,0.013770
34,LAFAYETTE,0.014451
44,MONROE,0.013564
60,SUMTER,0.014013
65,WAKULLA,0.012262


In [16]:
median = florida_dropout_rate["Total Dropout Rate"].median()
median

0.011628

In [86]:
florida_dropout_rate_median = florida_dropout_rate.loc[(florida_dropout_rate["Total Dropout Rate"] > median - .002) & (florida_dropout_rate["Total Dropout Rate"] < median + .003)]
districts = florida_dropout_rate_median["District Name"]
florida_dropout_rate_median

,District Name,Total Dropout Rate
2,BAKER,0.009673
3,BAY,0.011826
6,BROWARD,0.014282
10,CLAY,0.010588
11,COLLIER,0.010949
17,ESCAMBIA,0.010126
22,GLADES,0.011628
23,GULF,0.009820
29,HILLSBOROUGH,0.013770
34,LAFAYETTE,0.014451


# Determining the Districts for Analysis

In [87]:
florida_dropout_rate_median["Above/Below"] = np.where(florida_dropout_rate_median['Total Dropout Rate'] > median, 'Above', 'Below')
florida_dropout_rate_median

<ipython-input-87-91cb14078f91>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  florida_dropout_rate_median["Above/Below"] = np.where(florida_dropout_rate_median['Total Dropout Rate'] > median, 'Above', 'Below')


,District Name,Total Dropout Rate,Above/Below
2,BAKER,0.009673,Below
3,BAY,0.011826,Above
6,BROWARD,0.014282,Above
10,CLAY,0.010588,Below
11,COLLIER,0.010949,Below
17,ESCAMBIA,0.010126,Below
22,GLADES,0.011628,Below
23,GULF,0.009820,Below
29,HILLSBOROUGH,0.013770,Above
34,LAFAYETTE,0.014451,Above


In [73]:
# Creating List of Distric Names that are above the Median
florida_dropout_rate_df_above= florida_dropout_rate_median.loc[florida_dropout_rate_median["Total Dropout Rate"] > median]
districts_above_median = florida_dropout_rate_df_above["District Name"]
districts_above_median

3              BAY
6          BROWARD
29    HILLSBOROUGH
34       LAFAYETTE
44          MONROE
60          SUMTER
65         WAKULLA
Name: District Name, dtype: object

In [25]:
# Creating List of Distric Names that are below the Median
florida_dropout_rate_df_below = florida_dropout_rate_median.loc[florida_dropout_rate_median["Total Dropout Rate"] < median]
districts_below_median = florida_dropout_rate_df_below["District Name"]
districts_below_median

2         BAKER
10         CLAY
11      COLLIER
17     ESCAMBIA
23         GULF
46     OKALOOSA
56    ST. LUCIE
58     SARASOTA
64      VOLUSIA
Name: District Name, dtype: object

# Financial Analysis

In [92]:
# Pulling all financial data for districts for analysis
fl_district_financials = pd.merge(florida_dropout_rate_median,fl_nces_combined, on="District Name")
districts_financials = fl_district_financials[fl_district_financials["District Name"].isin(districts)]
districts_financials

,District Name,Total Dropout Rate,Above/Below,NCES ID,State ID,Degree of urbanization,Agency Type,Total Schools,Total Students Enrolled,Special Ed Students,...,Total Federal Revenue,Total State Revenue,Total Local Revenue,Local Revenue (Property Tax),Total Expense,Total Expense for Instruction Staff,Total Expense for General Admin,Total Expense for School Admin,Total Expense for Operations,Total Expense for Student Transportation
0,BAKER,0.009673,Below,1200060,02,32,1,9,4936,538,...,4704000.0,2.919700e+07,8773000,6266000,44360000,2595000,732000,2007000,4385000,2613000
1,BAY,0.011826,Above,1200090,03,22,1,51,27641,4575,...,26772000.0,1.035270e+08,137832000,101316000,278559000,11183000,2045000,13550000,20567000,8132000
2,BROWARD,0.014282,Above,1200180,06,21,1,382,266265,33535,...,285351000.0,9.802860e+08,1268177000,1067157000,2561097000,105513000,17827000,130535000,235664000,81193000
3,CLAY,0.010588,Below,1200300,10,21,1,50,35835,6768,...,25957000.0,1.888790e+08,91920000,65101000,302332000,16765000,2161000,14978000,24186000,10500000
4,COLLIER,0.010949,Below,1200330,11,21,1,72,45228,5775,...,53875000.0,8.222200e+07,399374000,362886000,534047000,29000000,3366000,29918000,51092000,18890000
5,ESCAMBIA,0.010126,Below,1200510,17,21,1,75,40735,5779,...,52213000.0,1.837050e+08,157076000,115995000,409375000,27568000,3502000,15570000,38989000,16275000
6,GLADES,0.011628,Below,1200660,22,41,1,11,1603,265,...,2248000.0,9.289000e+06,5220000,4132000,31237000,812000,583000,730000,1539000,515000
7,GULF,0.009820,Below,1200690,23,41,1,6,1872,308,...,2137000.0,6.173000e+06,10931000,9529000,19561000,1471000,853000,831000,2332000,902000
8,HILLSBOROUGH,0.013770,Above,1200870,29,21,1,329,207469,28985,...,299615000.0,1.022546e+09,687916000,530555000,2165489000,157221000,13538000,102680000,139813000,74337000
9,LAFAYETTE,0.014451,Above,1201020,34,43,1,7,1244,184,...,1514000.0,7.495000e+06,2484000,1848000,11127000,1067000,559000,601000,939000,541000


In [33]:
# Pulling all financial data for districts below the median
districts_below_financials = fl_nces_combined[fl_nces_combined["District Name"].isin(districts_below_median)]
districts_below_financials

,District Name,NCES ID,State ID,Degree of urbanization,Agency Type,Total Schools,Total Students Enrolled,Special Ed Students,English Language Students,Total FTE Teachers,...,Total Federal Revenue,Total State Revenue,Total Local Revenue,Local Revenue (Property Tax),Total Expense,Total Expense for Instruction Staff,Total Expense for General Admin,Total Expense for School Admin,Total Expense for Operations,Total Expense for Student Transportation
1,BAKER,1200060,02,32,1,9,4936,538,11,301,...,4704000.0,29197000.0,8773000,6266000,44360000,2595000,732000,2007000,4385000,2613000
9,CLAY,1200300,10,21,1,50,35835,6768,609,2462,...,25957000.0,188879000.0,91920000,65101000,302332000,16765000,2161000,14978000,24186000,10500000
10,COLLIER,1200330,11,21,1,72,45228,5775,5823,2954,...,53875000.0,82222000.0,399374000,362886000,534047000,29000000,3366000,29918000,51092000,18890000
16,ESCAMBIA,1200510,17,21,1,75,40735,5779,466,2778,...,52213000.0,183705000.0,157076000,115995000,409375000,27568000,3502000,15570000,38989000,16275000
22,GULF,1200690,23,41,1,6,1872,308,8,130,...,2137000.0,6173000.0,10931000,9529000,19561000,1471000,853000,831000,2332000,902000
45,OKALOOSA,1201380,46,22,1,54,30415,4071,944,1824,...,27629000.0,120964000.0,133196000,110488000,291117000,11574000,2488000,19393000,24307000,12203000
55,SARASOTA,1201680,58,13,1,64,41912,6217,2550,3015,...,38826000.0,75732000.0,396452000,347995000,533998000,18612000,4095000,17863000,48387000,16210000
58,ST. LUCIE,1201770,56,12,1,53,40074,4616,3070,2387,...,45043000.0,173542000.0,174135000,120677000,376538000,18167000,4829000,22147000,34129000,21532000
63,VOLUSIA,1201920,64,21,1,92,61777,9879,3479,4153,...,69002000.0,255682000.0,270472000,203540000,576168000,37538000,4455000,36135000,50429000,16515000


# Revenue Analaysis

In [94]:
district_rev = districts_financials[["District Name","Above/Below", "Total Schools", "Total Students Enrolled", "Total Revenue",
                                             "Total Federal Revenue", "Total State Revenue", "Total Local Revenue",
                                            "Local Revenue (Property Tax)"]]


perc_fed = district_rev["Total Federal Revenue"] / district_rev["Total Revenue"]
perc_state = district_rev["Total State Revenue"]/district_rev["Total Revenue"]
perc_local = district_rev["Total Local Revenue"]/district_rev["Total Revenue"]
perc_prop_tax = district_rev["Local Revenue (Property Tax)"]/district_rev["Total Local Revenue"]


district_rev["Revenue % from Federal"] = perc_fed
district_rev["Revenue % from State"] = perc_state
district_rev["Revenue % from Local"] = perc_local
district_rev["Revenue % from Property Tax"] = perc_prop_tax

# Format columns
district_rev = district_rev.style.format({"Total Revenue": "${:,.0f}" , "Total Federal Revenue": "${:,.0f}",
                                                            "Total State Revenue":"${:,.0f}", "Total Local Revenue":"${:,.0f}",
                                                            "Local Revenue (Property Tax)":"${:,.0f}", 
                                                            "Revenue % from Federal":"{:.2%}", "Revenue % from State":"{:.2%}",
                                                            "Revenue % from Local":"{:.2%}", "Revenue % from Property Tax":"{:.2%}"})

district_rev

<ipython-input-94-edf7daa2b28f>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  district_rev["Revenue % from Federal"] = perc_fed
<ipython-input-94-edf7daa2b28f>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  district_rev["Revenue % from State"] = perc_state
<ipython-input-94-edf7daa2b28f>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org

,District Name,Above/Below,Total Schools,Total Students Enrolled,Total Revenue,Total Federal Revenue,Total State Revenue,Total Local Revenue,Local Revenue (Property Tax),Revenue % from Federal,Revenue % from State,Revenue % from Local,Revenue % from Property Tax
0,BAKER,Below,9,4936,"$42,674,000","$4,704,000","$29,197,000","$8,773,000","$6,266,000",11.02%,68.42%,20.56%,71.42%
1,BAY,Above,51,27641,"$268,131,000","$26,772,000","$103,527,000","$137,832,000","$101,316,000",9.98%,38.61%,51.40%,73.51%
2,BROWARD,Above,382,266265,"$2,533,814,000","$285,351,000","$980,286,000","$1,268,177,000","$1,067,157,000",11.26%,38.69%,50.05%,84.15%
3,CLAY,Below,50,35835,"$306,756,000","$25,957,000","$188,879,000","$91,920,000","$65,101,000",8.46%,61.57%,29.97%,70.82%
4,COLLIER,Below,72,45228,"$535,471,000","$53,875,000","$82,222,000","$399,374,000","$362,886,000",10.06%,15.36%,74.58%,90.86%
5,ESCAMBIA,Below,75,40735,"$392,994,000","$52,213,000","$183,705,000","$157,076,000","$115,995,000",13.29%,46.74%,39.97%,73.85%
6,GLADES,Below,11,1603,"$16,757,000","$2,248,000","$9,289,000","$5,220,000","$4,132,000",13.42%,55.43%,31.15%,79.16%
7,GULF,Below,6,1872,"$19,241,000","$2,137,000","$6,173,000","$10,931,000","$9,529,000",11.11%,32.08%,56.81%,87.17%
8,HILLSBOROUGH,Above,329,207469,"$2,010,077,000","$299,615,000","$1,022,546,000","$687,916,000","$530,555,000",14.91%,50.87%,34.22%,77.12%
9,LAFAYETTE,Above,7,1244,"$11,493,000","$1,514,000","$7,495,000","$2,484,000","$1,848,000",13.17%,65.21%,21.61%,74.40%


In [35]:

dist_above_rev = districts_above_financials[["District Name","Total Schools", "Total Students Enrolled", "Total Revenue",
                                             "Total Federal Revenue", "Total State Revenue", "Total Local Revenue",
                                            "Local Revenue (Property Tax)"]]


above_perc_fed = dist_above_rev["Total Federal Revenue"] / dist_above_rev["Total Revenue"]
above_perc_state = dist_above_rev["Total State Revenue"]/dist_above_rev["Total Revenue"]
above_perc_local = dist_above_rev["Total Local Revenue"]/dist_above_rev["Total Revenue"]
above_perc_prop_tax = dist_above_rev["Local Revenue (Property Tax)"]/dist_above_rev["Total Local Revenue"]


dist_above_rev["Revenue % from Federal"] = above_perc_fed
dist_above_rev["Revenue % from State"] = above_perc_state
dist_above_rev["Revenue % from Local"] = above_perc_local
dist_above_rev["Revenue % from Property Tax"] = above_perc_prop_tax

# Format columns
dist_above_rev = dist_above_rev.style.format({"Total Revenue": "${:,.0f}" , "Total Federal Revenue": "${:,.0f}",
                                                            "Total State Revenue":"${:,.0f}", "Total Local Revenue":"${:,.0f}",
                                                            "Local Revenue (Property Tax)":"${:,.0f}", 
                                                            "Revenue % from Federal":"{:.2%}", "Revenue % from State":"{:.2%}",
                                                            "Revenue % from Local":"{:.2%}", "Revenue % from Property Tax":"{:.2%}"})

dist_above_rev

<ipython-input-35-213bf5bc8e36>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dist_above_rev["Revenue % from Federal"] = above_perc_fed
<ipython-input-35-213bf5bc8e36>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dist_above_rev["Revenue % from State"] = above_perc_state
<ipython-input-35-213bf5bc8e36>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://p

,District Name,Total Schools,Total Students Enrolled,Total Revenue,Total Federal Revenue,Total State Revenue,Total Local Revenue,Local Revenue (Property Tax),Revenue % from Federal,Revenue % from State,Revenue % from Local,Revenue % from Property Tax
2,BAY,51,27641,"$268,131,000","$26,772,000","$103,527,000","$137,832,000","$101,316,000",9.98%,38.61%,51.40%,73.51%
5,BROWARD,382,266265,"$2,533,814,000","$285,351,000","$980,286,000","$1,268,177,000","$1,067,157,000",11.26%,38.69%,50.05%,84.15%
28,HILLSBOROUGH,329,207469,"$2,010,077,000","$299,615,000","$1,022,546,000","$687,916,000","$530,555,000",14.91%,50.87%,34.22%,77.12%
33,LAFAYETTE,7,1244,"$11,493,000","$1,514,000","$7,495,000","$2,484,000","$1,848,000",13.17%,65.21%,21.61%,74.40%
43,MONROE,26,8478,"$124,086,000","$10,684,000","$15,022,000","$98,380,000","$75,804,000",8.61%,12.11%,79.28%,77.05%
59,SUMTER,14,8307,"$82,201,000","$7,445,000","$15,645,000","$59,111,000","$56,896,000",9.06%,19.03%,71.91%,96.25%
64,WAKULLA,15,5141,"$44,700,000","$4,754,000","$28,484,000","$11,462,000","$9,449,000",10.64%,63.72%,25.64%,82.44%


In [39]:
dist_below_rev = districts_below_financials[["District Name","Total Schools", "Total Students Enrolled", "Total Revenue",
                                             "Total Federal Revenue", "Total State Revenue", "Total Local Revenue",
                                            "Local Revenue (Property Tax)"]]


below_perc_fed = dist_below_rev["Total Federal Revenue"] / dist_below_rev["Total Revenue"]
below_perc_state = dist_below_rev["Total State Revenue"]/dist_below_rev["Total Revenue"]
below_perc_local = dist_below_rev["Total Local Revenue"]/dist_below_rev["Total Revenue"]
below_perc_prop_tax = dist_below_rev["Local Revenue (Property Tax)"]/dist_below_rev["Total Local Revenue"]


dist_below_rev["Revenue % from Federal"] = below_perc_fed
dist_below_rev["Revenue % from State"] = below_perc_state
dist_below_rev["Revenue % from Local"] = below_perc_local
dist_below_rev["Revenue % from Property Tax"] = below_perc_prop_tax

# Format columns
dist_below_rev = dist_below_rev.style.format({"Total Revenue": "${:,.0f}" , "Total Federal Revenue": "${:,.0f}",
                                                            "Total State Revenue":"${:,.0f}", "Total Local Revenue":"${:,.0f}",
                                                            "Local Revenue (Property Tax)":"${:,.0f}", 
                                                            "Revenue % from Federal":"{:.2%}", "Revenue % from State":"{:.2%}",
                                                            "Revenue % from Local":"{:.2%}", "Revenue % from Property Tax":"{:.2%}"})

dist_below_rev

<ipython-input-39-52caf4a5d646>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dist_below_rev["Revenue % from Federal"] = below_perc_fed
<ipython-input-39-52caf4a5d646>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dist_below_rev["Revenue % from State"] = below_perc_state
<ipython-input-39-52caf4a5d646>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://p

,District Name,Total Schools,Total Students Enrolled,Total Revenue,Total Federal Revenue,Total State Revenue,Total Local Revenue,Local Revenue (Property Tax),Revenue % from Federal,Revenue % from State,Revenue % from Local,Revenue % from Property Tax
1,BAKER,9,4936,"$42,674,000","$4,704,000","$29,197,000","$8,773,000","$6,266,000",11.02%,68.42%,20.56%,71.42%
9,CLAY,50,35835,"$306,756,000","$25,957,000","$188,879,000","$91,920,000","$65,101,000",8.46%,61.57%,29.97%,70.82%
10,COLLIER,72,45228,"$535,471,000","$53,875,000","$82,222,000","$399,374,000","$362,886,000",10.06%,15.36%,74.58%,90.86%
16,ESCAMBIA,75,40735,"$392,994,000","$52,213,000","$183,705,000","$157,076,000","$115,995,000",13.29%,46.74%,39.97%,73.85%
22,GULF,6,1872,"$19,241,000","$2,137,000","$6,173,000","$10,931,000","$9,529,000",11.11%,32.08%,56.81%,87.17%
45,OKALOOSA,54,30415,"$281,789,000","$27,629,000","$120,964,000","$133,196,000","$110,488,000",9.80%,42.93%,47.27%,82.95%
55,SARASOTA,64,41912,"$511,010,000","$38,826,000","$75,732,000","$396,452,000","$347,995,000",7.60%,14.82%,77.58%,87.78%
58,ST. LUCIE,53,40074,"$392,720,000","$45,043,000","$173,542,000","$174,135,000","$120,677,000",11.47%,44.19%,44.34%,69.30%
63,VOLUSIA,92,61777,"$595,156,000","$69,002,000","$255,682,000","$270,472,000","$203,540,000",11.59%,42.96%,45.45%,75.25%


# Expense Analaysis

In [95]:
distict_exp = districts_financials[["District Name","Above/Below", "Total Schools", "Total Students Enrolled", "Total FTE Teachers",
                                             "Total Expense", "Total Expense for Instruction Staff", "Total Expense for General Admin", 
                                             "Total Expense for School Admin", "Total Expense for Operations",
                                             "Total Expense for Student Transportation"]]


perc_instr = distict_exp["Total Expense for Instruction Staff"] / distict_exp["Total Expense"]
perc_gen_admin = distict_exp["Total Expense for General Admin"]/distict_exp["Total Expense"]
perc_sch_admin = distict_exp["Total Expense for School Admin"]/distict_exp["Total Expense"]
perc_oper = distict_exp["Total Expense for Operations"]/distict_exp["Total Expense"]
perc_trans = distict_exp["Total Expense for Student Transportation"]/distict_exp["Total Expense"]
per_school = distict_exp["Total Expense"]/distict_exp["Total Schools"]
per_student = distict_exp["Total Expense"]/distict_exp["Total Students Enrolled"]
per_teacher = distict_exp["Total Expense"]/distict_exp["Total FTE Teachers"]


distict_exp["Expense % for Instruction Staff"] = perc_instr
distict_exp["Expense % for General Admin"] = perc_gen_admin
distict_exp["Expense % for School Admin"] = perc_sch_admin
distict_exp["Expense % for Operations"] = perc_oper
distict_exp["Expense % for Transportation"] = perc_trans
distict_exp["Expense per School"] = per_school
distict_exp["Expense per Student"] = per_student
distict_exp["Expense per Teacher"] = per_teacher


# Format columns
distict_exp = distict_exp.style.format({"Total Expense": "${:,.0f}" , "Total Expense for Instruction Staff":"${:,.0f}",
                                                            "Total Expense for General Admin":"${:,.0f}", "Total Expense for School Admin":"${:,.0f}",
                                                            "Total Expense for Operations":"${:,.0f}", "Total Expense for Student Transportation":"${:,.0f}",
                                                            "Expense % for Instruction Staff":"{:.2%}", "Expense % for General Admin":"{:.2%}",
                                                            "Expense % for School Admin":"{:.2%}", "Expense % for Operations":"{:.2%}",
                                                            "Expense % for Transportation":"{:.2%}" ,"Expense per School":"${:,.0f}",
                                                            "Expense per Student":"${:,.0f}", "Expense per Teacher":"${:,.0f}"})

distict_exp

<ipython-input-95-574a0e857606>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  distict_exp["Expense % for Instruction Staff"] = perc_instr
<ipython-input-95-574a0e857606>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  distict_exp["Expense % for General Admin"] = perc_gen_admin
<ipython-input-95-574a0e857606>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https

,District Name,Above/Below,Total Schools,Total Students Enrolled,Total FTE Teachers,Total Expense,Total Expense for Instruction Staff,Total Expense for General Admin,Total Expense for School Admin,Total Expense for Operations,Total Expense for Student Transportation,Expense % for Instruction Staff,Expense % for General Admin,Expense % for School Admin,Expense % for Operations,Expense % for Transportation,Expense per School,Expense per Student,Expense per Teacher
0,BAKER,Below,9,4936,301,"$44,360,000","$2,595,000","$732,000","$2,007,000","$4,385,000","$2,613,000",5.85%,1.65%,4.52%,9.89%,5.89%,"$4,928,889","$8,987","$147,375"
1,BAY,Above,51,27641,1844,"$278,559,000","$11,183,000","$2,045,000","$13,550,000","$20,567,000","$8,132,000",4.01%,0.73%,4.86%,7.38%,2.92%,"$5,461,941","$10,078","$151,062"
2,BROWARD,Above,382,266265,15210,"$2,561,097,000","$105,513,000","$17,827,000","$130,535,000","$235,664,000","$81,193,000",4.12%,0.70%,5.10%,9.20%,3.17%,"$6,704,442","$9,619","$168,382"
3,CLAY,Below,50,35835,2462,"$302,332,000","$16,765,000","$2,161,000","$14,978,000","$24,186,000","$10,500,000",5.55%,0.71%,4.95%,8.00%,3.47%,"$6,046,640","$8,437","$122,799"
4,COLLIER,Below,72,45228,2954,"$534,047,000","$29,000,000","$3,366,000","$29,918,000","$51,092,000","$18,890,000",5.43%,0.63%,5.60%,9.57%,3.54%,"$7,417,319","$11,808","$180,788"
5,ESCAMBIA,Below,75,40735,2778,"$409,375,000","$27,568,000","$3,502,000","$15,570,000","$38,989,000","$16,275,000",6.73%,0.86%,3.80%,9.52%,3.98%,"$5,458,333","$10,050","$147,363"
6,GLADES,Below,11,1603,116,"$31,237,000","$812,000","$583,000","$730,000","$1,539,000","$515,000",2.60%,1.87%,2.34%,4.93%,1.65%,"$2,839,727","$19,487","$269,284"
7,GULF,Below,6,1872,130,"$19,561,000","$1,471,000","$853,000","$831,000","$2,332,000","$902,000",7.52%,4.36%,4.25%,11.92%,4.61%,"$3,260,167","$10,449","$150,469"
8,HILLSBOROUGH,Above,329,207469,16788,"$2,165,489,000","$157,221,000","$13,538,000","$102,680,000","$139,813,000","$74,337,000",7.26%,0.63%,4.74%,6.46%,3.43%,"$6,582,033","$10,438","$128,990"
9,LAFAYETTE,Above,7,1244,68,"$11,127,000","$1,067,000","$559,000","$601,000","$939,000","$541,000",9.59%,5.02%,5.40%,8.44%,4.86%,"$1,589,571","$8,945","$163,632"


In [67]:
dist_above_exp = districts_above_financials[["District Name","Total Schools", "Total Students Enrolled", "Total FTE Teachers",
                                             "Total Expense", "Total Expense for Instruction Staff", "Total Expense for General Admin", 
                                             "Total Expense for School Admin", "Total Expense for Operations",
                                             "Total Expense for Student Transportation"]]


above_perc_instr = dist_above_exp["Total Expense for Instruction Staff"] / dist_above_exp["Total Expense"]
above_perc_gen_admin = dist_above_exp["Total Expense for General Admin"]/dist_above_exp["Total Expense"]
above_perc_sch_admin = dist_above_exp["Total Expense for School Admin"]/dist_above_exp["Total Expense"]
above_perc_oper = dist_above_exp["Total Expense for Operations"]/dist_above_exp["Total Expense"]
above_perc_trans = dist_above_exp["Total Expense for Student Transportation"]/dist_above_exp["Total Expense"]
above_per_school = dist_above_exp["Total Expense"]/dist_above_exp["Total Schools"]
above_per_student = dist_above_exp["Total Expense"]/dist_above_exp["Total Students Enrolled"]
above_per_teacher = dist_above_exp["Total Expense"]/dist_above_exp["Total FTE Teachers"]


dist_above_exp["Expense % for Instruction Staff"] = above_perc_instr
dist_above_exp["Expense % for General Admin"] = above_perc_gen_admin
dist_above_exp["Expense % for School Admin"] = above_perc_sch_admin
dist_above_exp["Expense % for Operations"] = above_perc_oper
dist_above_exp["Expense % for Transportation"] = above_perc_trans
dist_above_exp["Expense per School"] = above_per_school
dist_above_exp["Expense per Student"] = above_per_student
dist_above_exp["Expense per Teacher"] = above_per_teacher


# Format columns
dist_above_exp = dist_above_exp.style.format({"Total Expense": "${:,.0f}" , "Total Expense for Instruction Staff":"${:,.0f}",
                                                            "Total Expense for General Admin":"${:,.0f}", "Total Expense for School Admin":"${:,.0f}",
                                                            "Total Expense for Operations":"${:,.0f}", "Total Expense for Student Transportation":"${:,.0f}",
                                                            "Expense % for Instruction Staff":"{:.2%}", "Expense % for General Admin":"{:.2%}",
                                                            "Expense % for School Admin":"{:.2%}", "Expense % for Operations":"{:.2%}",
                                                            "Expense % for Transportation":"{:.2%}" ,"Expense per School":"${:,.0f}",
                                                            "Expense per Student":"${:,.0f}", "Expense per Teacher":"${:,.0f}"})

dist_above_exp

<ipython-input-67-17e13a26aca2>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dist_above_exp["Expense % for Instruction Staff"] = above_perc_instr
<ipython-input-67-17e13a26aca2>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dist_above_exp["Expense % for General Admin"] = above_perc_gen_admin
<ipython-input-67-17e13a26aca2>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the do

,District Name,Total Schools,Total Students Enrolled,Total FTE Teachers,Total Expense,Total Expense for Instruction Staff,Total Expense for General Admin,Total Expense for School Admin,Total Expense for Operations,Total Expense for Student Transportation,Expense % for Instruction Staff,Expense % for General Admin,Expense % for School Admin,Expense % for Operations,Expense % for Transportation,Expense per School,Expense per Student,Expense per Teacher
2,BAY,51,27641,1844,"$278,559,000","$11,183,000","$2,045,000","$13,550,000","$20,567,000","$8,132,000",4.01%,0.73%,4.86%,7.38%,2.92%,"$5,461,941","$10,078","$151,062"
5,BROWARD,382,266265,15210,"$2,561,097,000","$105,513,000","$17,827,000","$130,535,000","$235,664,000","$81,193,000",4.12%,0.70%,5.10%,9.20%,3.17%,"$6,704,442","$9,619","$168,382"
28,HILLSBOROUGH,329,207469,16788,"$2,165,489,000","$157,221,000","$13,538,000","$102,680,000","$139,813,000","$74,337,000",7.26%,0.63%,4.74%,6.46%,3.43%,"$6,582,033","$10,438","$128,990"
33,LAFAYETTE,7,1244,68,"$11,127,000","$1,067,000","$559,000","$601,000","$939,000","$541,000",9.59%,5.02%,5.40%,8.44%,4.86%,"$1,589,571","$8,945","$163,632"
43,MONROE,26,8478,562,"$108,025,000","$5,621,000","$1,378,000","$4,381,000","$9,187,000","$3,364,000",5.20%,1.28%,4.06%,8.50%,3.11%,"$4,154,808","$12,742","$192,215"
59,SUMTER,14,8307,546,"$82,264,000","$5,284,000","$1,172,000","$2,610,000","$5,471,000","$2,662,000",6.42%,1.42%,3.17%,6.65%,3.24%,"$5,876,000","$9,903","$150,667"
64,WAKULLA,15,5141,327,"$45,316,000","$1,768,000","$1,243,000","$2,650,000","$5,404,000","$2,823,000",3.90%,2.74%,5.85%,11.93%,6.23%,"$3,021,067","$8,815","$138,581"


In [70]:
dist_below_exp = districts_below_financials[["District Name","Total Schools", "Total Students Enrolled", "Total FTE Teachers", 
                                             "Total Expense","Total Expense for Instruction Staff", "Total Expense for General Admin", 
                                             "Total Expense for School Admin", "Total Expense for Operations",
                                             "Total Expense for Student Transportation"]]


below_perc_instr = dist_below_exp["Total Expense for Instruction Staff"] / dist_below_exp["Total Expense"]
below_perc_gen_admin = dist_below_exp["Total Expense for General Admin"]/dist_below_exp["Total Expense"]
below_perc_sch_admin = dist_below_exp["Total Expense for School Admin"]/dist_below_exp["Total Expense"]
below_perc_oper = dist_below_exp["Total Expense for Operations"]/dist_below_exp["Total Expense"]
below_perc_trans = dist_below_exp["Total Expense for Student Transportation"]/dist_below_exp["Total Expense"]
below_per_school = dist_below_exp["Total Expense"]/dist_below_exp["Total Schools"]
below_per_student = dist_below_exp["Total Expense"]/dist_below_exp["Total Students Enrolled"]
below_per_teacher = dist_below_exp["Total Expense"]/dist_below_exp["Total FTE Teachers"]


dist_below_exp["Expense % for Instruction Staff"] = below_perc_instr
dist_below_exp["Expense % for General Admin"] = below_perc_gen_admin
dist_below_exp["Expense % for School Admin"] = below_perc_sch_admin
dist_below_exp["Expense % for Operations"] = below_perc_oper
dist_below_exp["Expense % for Transportation"] = below_perc_trans
dist_below_exp["Expense per School"] = below_per_school
dist_below_exp["Expense per Student"] = below_per_student
dist_below_exp["Expense per Teacher"] = below_per_teacher

# Format columns
dist_below_exp = dist_below_exp.style.format({"Total Expense": "${:,.0f}" , "Total Expense for Instruction Staff":"${:,.0f}",
                                                            "Total Expense for General Admin":"${:,.0f}", "Total Expense for School Admin":"${:,.0f}",
                                                            "Total Expense for Operations":"${:,.0f}", "Total Expense for Student Transportation":"${:,.0f}",
                                                            "Expense % for Instruction Staff":"{:.2%}", "Expense % for General Admin":"{:.2%}",
                                                            "Expense % for School Admin":"{:.2%}", "Expense % for Operations":"{:.2%}",
                                                            "Expense % for Transportation":"{:.2%}" ,"Expense per School":"${:,.0f}",
                                                            "Expense per Student":"${:,.0f}", "Expense per Teacher":"${:,.0f}"})

dist_below_exp

<ipython-input-70-90807a3788cf>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dist_below_exp["Expense % for Instruction Staff"] = below_perc_instr
<ipython-input-70-90807a3788cf>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dist_below_exp["Expense % for General Admin"] = below_perc_gen_admin
<ipython-input-70-90807a3788cf>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the do

,District Name,Total Schools,Total Students Enrolled,Total FTE Teachers,Total Expense,Total Expense for Instruction Staff,Total Expense for General Admin,Total Expense for School Admin,Total Expense for Operations,Total Expense for Student Transportation,Expense % for Instruction Staff,Expense % for General Admin,Expense % for School Admin,Expense % for Operations,Expense % for Transportation,Expense per School,Expense per Student,Expense per Teacher
1,BAKER,9,4936,301,"$44,360,000","$2,595,000","$732,000","$2,007,000","$4,385,000","$2,613,000",5.85%,1.65%,4.52%,9.89%,5.89%,"$4,928,889","$8,987","$147,375"
9,CLAY,50,35835,2462,"$302,332,000","$16,765,000","$2,161,000","$14,978,000","$24,186,000","$10,500,000",5.55%,0.71%,4.95%,8.00%,3.47%,"$6,046,640","$8,437","$122,799"
10,COLLIER,72,45228,2954,"$534,047,000","$29,000,000","$3,366,000","$29,918,000","$51,092,000","$18,890,000",5.43%,0.63%,5.60%,9.57%,3.54%,"$7,417,319","$11,808","$180,788"
16,ESCAMBIA,75,40735,2778,"$409,375,000","$27,568,000","$3,502,000","$15,570,000","$38,989,000","$16,275,000",6.73%,0.86%,3.80%,9.52%,3.98%,"$5,458,333","$10,050","$147,363"
22,GULF,6,1872,130,"$19,561,000","$1,471,000","$853,000","$831,000","$2,332,000","$902,000",7.52%,4.36%,4.25%,11.92%,4.61%,"$3,260,167","$10,449","$150,469"
45,OKALOOSA,54,30415,1824,"$291,117,000","$11,574,000","$2,488,000","$19,393,000","$24,307,000","$12,203,000",3.98%,0.85%,6.66%,8.35%,4.19%,"$5,391,056","$9,571","$159,604"
55,SARASOTA,64,41912,3015,"$533,998,000","$18,612,000","$4,095,000","$17,863,000","$48,387,000","$16,210,000",3.49%,0.77%,3.35%,9.06%,3.04%,"$8,343,719","$12,741","$177,114"
58,ST. LUCIE,53,40074,2387,"$376,538,000","$18,167,000","$4,829,000","$22,147,000","$34,129,000","$21,532,000",4.82%,1.28%,5.88%,9.06%,5.72%,"$7,104,491","$9,396","$157,745"
63,VOLUSIA,92,61777,4153,"$576,168,000","$37,538,000","$4,455,000","$36,135,000","$50,429,000","$16,515,000",6.52%,0.77%,6.27%,8.75%,2.87%,"$6,262,696","$9,327","$138,735"


In [ ]:
# Calculate the IQR and quantitatively determine if there are any potential outliers. 
    
# quartiles = florida_dropout_rate["Total Dropout Rate"].quantile([.25,.5,.75])
# lowerq = quartiles[0.25]
# upperq = quartiles[0.75]
# iqr = upperq-lowerq
# lower_bound = lowerq - (1.5*iqr)
# upper_bound = upperq + (1.5*iqr)

# # Generate a box plot of the final tumor volume of each mouse across four regimens of interest with Matplotlib
# plt.boxplot(florida_dropout_rate["Total Dropout Rate"], patch_artist=False,
#                 medianprops=dict(color="g"),boxprops=dict(color="b"))
# plt.title("Boxplot grouped by Drug Regimen")
# plt.ylabel('Tumor Volume (mm3)')
# plt.xlabel('Drug Regimen')
# plt.show()